In [ ]:
import transformers
import torch
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
pipeline = transformers.pipeline("text-generation",model=model_id,model_kwargs={"torch_dtype": torch.bfloat16}, device_map="auto",)
def promptModel(_user_, _system_='You are a helpful digital assistant.', max_tokens=256):
    messages = [{"role": "system", "content": _system_},{"role": "user",   "content": _user_}]
    prompt = pipeline.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    terminators = [pipeline.tokenizer.eos_token_id,pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")]
    outputs = pipeline(prompt,max_new_tokens=max_tokens,eos_token_id=terminators,
                       do_sample=True,temperature=0.6,top_p=0.9,)
    return outputs[0]["generated_text"][len(prompt):]

In [ ]:
#
# Prepare an article for processing by loading it, separating it into sentences, and doing basic cleaning
#
import os
_base_dir_ = '../../../data/2014_vast/MC1/News Articles'
os.listdir(_base_dir_ + '/Everyday News')
_txt_raw_     = open(_base_dir_ + '/Everyday News/343.txt', 'rb').read()
_txt_better_  = str(_txt_raw_, encoding='utf-8').replace('\r', '').split('\n')
_txt_         = []
for _line_ in _txt_better_:
    if _line_ == '' or _line_ == '\n' or _line_ == '\r': continue
    if '.' in _line_:
        for _sent_ in _line_.split('.'):
            _sent_ = _sent_.strip()
            if _sent_ == '' or _sent_ == '\n' or _sent_ == '\r' or _sent_ == '.': continue
            _txt_.append(_sent_ + '.')
    else:
        _txt_.append(_line_.strip())
print('\n'.join(_txt_))

In [ ]:
#
# Grammar is a problem with the articles -- let's have the model fix that up first
#
_grammar_ = []
for _sent_ in _txt_:
    if _sent_.lower().startswith('source:') or _sent_.lower().startswith('title:') or _sent_.lower().startswith('published:'): continue
    _fixed_up_ = promptModel(_sent_, 'Make the following sentence grammatically correct.  Just return the corrected sentence with no explanation.')    
    _grammar_.append(_fixed_up_)
print('\n'.join(_grammar_))

In [ ]:
#
#
#
_entity_prompt_ = '''Extract the people, groups, organizations, countries, cities, places, and locations from the following sentence.
Return as CSV with a header of "entity, type".  Just return the CSV.  Do not include an explanation or any caveats.'''
i = 7
_example_ = promptModel(_grammar_[i], _entity_prompt_)
print(_grammar_[i])
print(_example_)